## Inital Setup and Data Import

In [192]:
import numpy as np
import pandas as pd
import re as re
import pickle

from sklearn.preprocessing import Imputer

import matplotlib.pyplot as plt
import seaborn as sns
import time
import scipy.stats as st
import scipy.special as spec

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit,StratifiedKFold, RandomizedSearchCV, GridSearchCV

from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, precision_score, recall_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

import math
from math import log

from xgboost import XGBClassifier

In [193]:
import warnings
warnings.filterwarnings("ignore")

import nbimporter
from libraries_setup import AllLibraries
from setup_code import *

AllLibraries()

In [194]:
data = pd.read_csv('train.csv')
scoring = pd.read_csv('test.csv')

full_data = [data, scoring]

In [195]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [196]:
data.set_index('PassengerId', inplace=True)

In [197]:
y = data.loc[:,'Survived']
X = data.drop ('Survived', 1)

In [198]:
X.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [199]:
random = 1992
test_prc = 0.2

X_train , X_test, y_train, y_test = train_test_split( X, y, test_size = test_prc, random_state = random)

In [200]:
data_train = X_train
data_train['Survived'] = y_train

data_test = X_test
data_test['Survived'] = y_test

In [201]:
data_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
PassengerId,,,,,,,,,,,
780,1,"Robert, Mrs. Edward Scott (Elisabeth Walton Mc...",female,43.0,0,1,24160,211.3375,B3,S,1
828,2,"Mallet, Master. Andre",male,1.0,0,2,S.C./PARIS 2079,37.0042,NaN,C,1
887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
110,3,"Moran, Miss. Bertha",female,NaN,1,0,371110,24.1500,NaN,Q,1
393,3,"Gustafsson, Mr. Johan Birger",male,28.0,2,0,3101277,7.9250,NaN,S,0


In [205]:
#train, calibration = train_test_split(data_train, test_size=0.2)

In [204]:
#ShowTypes(data_train)
ShowLevels(data_train)

Number of records in the dataset is  712
 
Number of  Name  levels:  712
Number of  Sex  levels:  2
Number of  Ticket  levels:  564
Number of  Cabin  levels:  130
Number of  Embarked  levels:  3
Number of  Title  levels:  7
Number of  Last_name  levels:  558
Number of  Other_full_name  levels:  119
Number of  Other_last_name  levels:  117
Number of  Cabin_letter  levels:  7
Number of  Ticket_first  levels:  37
Number of  Ticket_second  levels:  554


In [203]:
############### FEATURE ENGINEERING ####################
create_new_var (data_train)

#Embarked
data_train = impute_most_frequent (data_train, 'Embarked')

In [70]:
#data_train = impact_coding(data_train,data_train, 'Embarked', 'Embarked_impact')
[data_train, log_cabin] = impact_coding(data_train, 'Cabin', 'Cabin_impact')
[data_train, log_ticket] = impact_coding(data_train, 'Ticket', 'Ticket_impact')
[data_train, log_last_name] = impact_coding(data_train, 'Last_name', 'Last_name_impact')
[data_train, log_other_last_name] = impact_coding(data_train, 'Other_last_name', 'Other_last_name_impact')

[data_train, log_ticket_first] = impact_coding(data_train, 'Ticket_first', 'Ticket_first_impact')
[data_train, log_ticket_second] = impact_coding(data_train, 'Ticket_second', 'Ticket_second_impact')

data_train = change_var_types (data_train)

In [71]:
#Age
add_missing_ind(data_train, 'Age', 'MisAge')
data_train = impute_regressor(data_train, 'Age')

In [72]:
data_train = data_train.drop ('Name', 1) # 1 for columns, 0 for rows
data_train = data_train.drop ('Ticket', 1)
data_train = data_train.drop ('Cabin', 1) 

data_train = data_train.drop ('Last_name', 1) 
data_train = data_train.drop ('Other_full_name', 1) 
data_train = data_train.drop ('Other_last_name', 1) 
data_train = data_train.drop ('Ticket_first', 1) 
data_train = data_train.drop ('Ticket_second', 1) 

In [73]:
data_train = create_dummies(data_train,0)

In [75]:
############### FEATURE ENGINEERING FOR TESTING ####################

create_new_var (data_test)

#Embarked
data_test = impute_most_frequent (data_test, 'Embarked')

data_test = pd.merge(data_test, log_cabin,  on='Cabin', how = 'left')
data_test['Cabin_impact'] = data_test['Cabin_impact'].fillna(0)

data_test = pd.merge(data_test, log_ticket,  on='Ticket', how = 'left')
data_test['Ticket_impact'] = data_test['Ticket_impact'].fillna(0)

data_test = pd.merge(data_test, log_last_name,  on='Last_name', how = 'left')
data_test['Last_name_impact'] = data_test['Last_name_impact'].fillna(0)

data_test = pd.merge(data_test, log_other_last_name,  on='Other_last_name', how = 'left')
data_test['Other_last_name_impact'] = data_test['Other_last_name_impact'].fillna(0)

data_test = pd.merge(data_test, log_ticket_first,  on='Ticket_first', how = 'left')
data_test['Ticket_first_impact'] = data_test['Ticket_first_impact'].fillna(0)

data_test = pd.merge(data_test, log_ticket_second,  on='Ticket_second', how = 'left')
data_test['Ticket_second_impact'] = data_test['Ticket_second_impact'].fillna(0)

data_test = change_var_types (data_test)

#Age
add_missing_ind(data_test, 'Age', 'MisAge')
impute_regressor(data_test, 'Age')

data_test = data_test.drop ('Name', 1) # 1 for columns, 0 for rows
data_test = data_test.drop ('Ticket', 1)
data_test = data_test.drop ('Cabin', 1) 

data_test = data_test.drop ('Last_name', 1) 
data_test = data_test.drop ('Other_full_name', 1) 
data_test = data_test.drop ('Other_last_name', 1) 
data_test = data_test.drop ('Ticket_first', 1) 
data_test = data_test.drop ('Ticket_second', 1) 

data_test = create_dummies(data_test,0)

In [76]:
#ShowTypes(data_train)
#data_train.isna().sum()
#ShowLevels(data_train)

 Categorical variables of the dataset are: 


 
 Numerical variables of the dataset are: 

Age
Fare
Survived
Cabin_impact
Ticket_impact
Last_name_impact
Other_last_name_impact
Ticket_first_impact
Ticket_second_impact


## Modeling

In [77]:
y_train = data_train.loc[:,'Survived']
X_train = data_train.drop ('Survived', 1)

y_test = data_test.loc[:,'Survived']
X_test = data_test.drop ('Survived', 1)

In [79]:
X_test.head()

,Age,Fare,Cabin_impact,Ticket_impact,Last_name_impact,Other_last_name_impact,Ticket_first_impact,Ticket_second_impact,Pclass_1,Pclass_2,...,Title_Rev,Cabin_letter_A,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_G,Cabin_letter_T,MisAge_0.0,MisAge_1.0
0,19.0,6.7500,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0,0,...,0,0,0,0,0,0,0,0,1,0
1,70.5,7.7500,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0,0,...,0,0,0,0,0,0,0,0,1,0
2,37.0,7.9250,0.0,0.000000,-9.464012,0.0,0.000000,0.000000,0,0,...,0,0,0,0,0,0,0,0,1,0
3,32.5,27.7208,0.0,0.000000,0.000000,0.0,1.104499,0.000000,1,0,...,0,0,0,0,0,0,0,0,0,1
4,13.0,19.5000,0.0,9.649966,9.649966,0.0,0.000000,9.649966,0,1,...,0,0,0,0,0,0,0,0,1,0


### Parameter Tuning

In [80]:
############################ TUNNING RANDOM FOREST ########################################

rf = RandomForestClassifier()
#rf.get_params()

row_num = X_train.shape[0]

n_estimators_start = 200
n_estimators_end =  2000

max_depth_start = 10
max_depth_end = 110

min_samples_split_start =  round(0.01*row_num) #2
min_samples_split_end = round(0.03*row_num) #10

min_samples_leaf_start = 1
min_samples_leaf_end = 20 #4


max_features = ['auto', 'sqrt']
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': st.randint (n_estimators_start, n_estimators_end),
               'max_depth': st.randint (max_depth_start, max_depth_end),
               'min_samples_split': st.randint (min_samples_split_start, min_samples_split_end),
               'min_samples_leaf': st.randint (min_samples_leaf_start, min_samples_leaf_end),
               'max_features': max_features,
               'bootstrap': bootstrap}


rf_random = RandomizedSearchCV(estimator = rf, 
                                           param_distributions = random_grid, 
                                           n_iter = 100, cv = 5, scoring = 'roc_auc', verbose=False, 
                                           random_state=42, n_jobs = -1)

start = time.time()
rf_random.fit(X_train,y_train)
end = time.time()

print("Execution Time Was: ", end - start, "seconds")

RandomForestClassifierTuned = RandomForestClassifier()
RandomForestClassifierTuned.set_params(**rf_random.best_params_)

##########################################################################################

Execution Time Was:  162.45745992660522 seconds


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=37, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=11,
            min_weight_fraction_leaf=0.0, n_estimators=1219, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [81]:
############################ TUNNING GRADIENT BOOSTING ########################################

#best model (roc = 82.727483): with n_est 3 - 40; max_depth 3-40; learning rate 0.05 - 0.4

#https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/
# consider using GridSearchCV

gb = GradientBoostingClassifier()
#rf.get_params()

row_num = X_train.shape[0]

learning_rate_start = 0.09 #0.03
learning_rate_end = 0.2 #0.3

n_estimators_start = 3
n_estimators_end =  40 #80

max_depth_start = 3
max_depth_end = 40 #20

min_samples_split_start = round(0.01*row_num)
min_samples_split_end = round(0.03*row_num)

min_samples_leaf_start = 1
min_samples_leaf_end = 20 


random_grid = {  
                "learning_rate": st.uniform (learning_rate_start, learning_rate_end),
                "n_estimators": st.randint (n_estimators_start, n_estimators_end),
                "max_depth": st.randint (max_depth_start, max_depth_end)
            }


# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
gb_random = RandomizedSearchCV(estimator = gb, 
                                           param_distributions = random_grid, 
                                           n_iter = 100, cv = 10, scoring = 'roc_auc', verbose=False, 
                                           random_state=42, n_jobs = -1)


start = time.time()
gb_random.fit(X_train,y_train)
end = time.time()

print("Execution Time Was: ", end - start, "seconds")

GradientBoostingClassifierTuned = GradientBoostingClassifier()
GradientBoostingClassifierTuned.set_params(**gb_random.best_params_)
GradientBoostingClassifierTuned.get_params()
############################################################################################

Execution Time Was:  6.960388898849487 seconds


{'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.2759395304685146,
 'loss': 'deviance',
 'max_depth': 8,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 30,
 'presort': 'auto',
 'random_state': None,
 'subsample': 1.0,
 'verbose': 0,
 'warm_start': False}

In [82]:
############################ TUNNING XGBOOST ########################################

#https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/
# consider using GridSearchCV

xgb = XGBClassifier()
#rf.get_params()

row_num = X_train.shape[0]

learning_rate_start = 0.05
learning_rate_end = 0.4

n_estimators_start = 3
n_estimators_end =  80

max_depth_start = 3
max_depth_end = 50

one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

random_grid = {  
    "n_estimators": st.randint(n_estimators_start, n_estimators_end),
    "max_depth": st.randint(max_depth_start, max_depth_end),
    "learning_rate": st.uniform(learning_rate_start, learning_rate_end),
    "colsample_bytree": one_to_left,
    "subsample": one_to_left,
    "gamma": st.uniform(0, 10),
    'reg_alpha': from_zero_positive,
    "min_child_weight": from_zero_positive
}

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
xgb_random = RandomizedSearchCV(estimator = xgb, 
                                           param_distributions = random_grid, 
                                           n_iter = 100, cv = 10, scoring = 'roc_auc', verbose=False, 
                                           random_state=42, n_jobs = -1)


start = time.time()
xgb_random.fit(X_train,y_train)
end = time.time()

print("Execution Time Was: ", end - start, "seconds")

XGBClassifierTuned = XGBClassifier()
XGBClassifierTuned.set_params(**xgb_random.best_params_)
XGBClassifierTuned.get_params()
#####################################################################################

Execution Time Was:  6.82409405708313 seconds


{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bytree': 0.9759664405348728,
 'gamma': 7.709113770642705,
 'learning_rate': 0.21881442382027494,
 'max_delta_step': 0,
 'max_depth': 4,
 'min_child_weight': 9.093085252463617,
 'missing': None,
 'n_estimators': 67,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 4.321413783592179,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': True,
 'subsample': 0.9697082498532124}

### Comparing Different Classifiers

In [83]:
classifiers = [
    KNeighborsClassifier(3),
  # SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
  #  GaussianNB(),
  #  LinearDiscriminantAnalysis(),
  #  QuadraticDiscriminantAnalysis(),
    XGBClassifier(),
    RandomForestClassifierTuned,
    GradientBoostingClassifierTuned,
    XGBClassifierTuned
]

names = [
        'KNeighborsClassifier',
   #     'Support Vector Machine',
        'NuSVC',
        'Decision Tree',
        'Random Forest',
        'AdaBoost',
        'Gradient Boosting',
   #     'Gaussian NB',
   #     'Linear Discriminant Analysis',
   #     'Quadratic Discriminant Analysis',
        'XGBoost',
        'Random Forest Tuned',
        'Gradient Boosting Tuned',
        'XGBoost Tuned'
]

In [84]:
folds = 10
random = 23
test_prc = 0.3

#X_train , X_test, y_train, y_test = train_test_split( X, y, test_size = test_prc, random_state = random)

sss = StratifiedShuffleSplit(n_splits = folds, test_size=test_prc, random_state=random)
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = random)


log_cols=["Index", "Classifier", "ROC", "Accuracy", "Log Loss", 'Precision', 'Recall']
log = pd.DataFrame(columns=log_cols)

index = 0

for train_index, test_index in skf.split(X_train, y_train):
    
    index += 1
    
    X_train_kf, X_test_kf = X_train.values[train_index], X_train.values[test_index]
    y_train_kf, y_test_kf = y_train.iloc[train_index], y_train.iloc[test_index]

    # FOR MORE METRICS SEE: http://scikit-learn.org/stable/modules/model_evaluation.html

    for name, clf in zip(names,classifiers):

        clf.fit(X_train_kf, y_train_kf)

        y_pred_kf = clf.predict(X_test_kf)
        y_pred_prob_kf = clf.predict_proba(X_test_kf)

        #name = clf.__class__.__name__

        roc = 100*roc_auc_score(y_test_kf, y_pred_kf)
        acc = 100*accuracy_score(y_test_kf, y_pred_kf)
        ll = log_loss(y_test_kf, y_pred_prob_kf)

        prec = 100*precision_score(y_test_kf, y_pred_kf)
        recall = 100*recall_score(y_test_kf, y_pred_kf)

        log_entry = pd.DataFrame([[index, name, roc, acc, ll, prec, recall]], columns=log_cols)
        log = log.append(log_entry)

#log
final_log = log.groupby(['Classifier'])["ROC", "Accuracy", "Log Loss", 'Precision', 'Recall'].mean()
final_log.sort_values(by = 'ROC', ascending = False, inplace = True)

In [191]:
final_log

,ROC,Accuracy,Log Loss,Precision,Recall
Classifier,,,,,
Random Forest,99.299305,99.301643,0.040318,98.988506,99.285714
Random Forest Tuned,99.069390,99.019953,0.046401,98.286535,99.285714
Gradient Boosting,99.066747,99.019953,0.035348,98.263547,99.285714
XGBoost,99.066747,99.019953,0.029970,98.263547,99.285714
AdaBoost,99.001812,99.017997,0.174245,98.608374,98.928571
XGBoost Tuned,98.896481,98.736307,0.090939,97.252053,99.642857
Decision Tree,98.825883,98.879108,0.387142,98.608374,98.571429
Gradient Boosting Tuned,98.709604,98.738263,0.078465,98.263547,98.571429
KNeighborsClassifier,97.010722,97.059859,0.334684,95.938964,96.785714


In [119]:
GradientBoostingClassifier().get_params()

{'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'presort': 'auto',
 'random_state': None,
 'subsample': 1.0,
 'verbose': 0,
 'warm_start': False}

In [120]:
GradientBoostingClassifierTuned.get_params()

{'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.24965903579335505,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'presort': 'auto',
 'random_state': None,
 'subsample': 1.0,
 'verbose': 0,
 'warm_start': False}

### Running The Model

In [125]:
# running on the whole dataset
best_classifier = RandomForestClassifier() #GradientBoostingClassifierTuned
best_classifier.fit(X_train, y_train)


y_pred = best_classifier.predict(X_test)
y_pred_prob = best_classifier.predict_proba(X_test)

#X_test['Survived_pred'] = y_pred
acc = 100*accuracy_score(y_test, y_pred)
acc

79.3296089385475

## Submitting to Kaggle

In [183]:
scoring = pd.read_csv('test.csv')

scoring.set_index('PassengerId', inplace=True)

create_new_var (scoring)

#Embarked
scoring = impute_most_frequent (scoring, 'Embarked')

scoring = scoring.reset_index().merge(log_cabin, on='Cabin', how = 'left').set_index('PassengerId')
scoring['Cabin_impact'] = scoring['Cabin_impact'].fillna(0)

scoring = scoring.reset_index().merge(log_ticket, on='Ticket', how = 'left').set_index('PassengerId')
scoring['Ticket_impact'] = scoring['Ticket_impact'].fillna(0)

scoring = scoring.reset_index().merge( log_last_name,  on='Last_name', how = 'left').set_index('PassengerId')
scoring['Last_name_impact'] = scoring['Last_name_impact'].fillna(0)

scoring = scoring.reset_index().merge(log_other_last_name,  on='Other_last_name', how = 'left').set_index('PassengerId')
scoring['Other_last_name_impact'] = scoring['Other_last_name_impact'].fillna(0)

scoring = scoring.reset_index().merge( log_ticket_first,  on='Ticket_first', how = 'left').set_index('PassengerId')
scoring['Ticket_first_impact'] = scoring['Ticket_first_impact'].fillna(0)

scoring = scoring.reset_index().merge( log_ticket_second,  on='Ticket_second', how = 'left').set_index('PassengerId')
scoring['Ticket_second_impact'] = scoring['Ticket_second_impact'].fillna(0)

scoring = change_var_types (scoring)

#Age
add_missing_ind(scoring, 'Age', 'MisAge')
impute_regressor(scoring, 'Age')
impute_regressor(scoring, 'Fare')

scoring = scoring.drop ('Name', 1) # 1 for columns, 0 for rows
scoring = scoring.drop ('Ticket', 1)
scoring = scoring.drop ('Cabin', 1) 


scoring = scoring.drop ('Last_name', 1) 
scoring = scoring.drop ('Other_full_name', 1) 
scoring = scoring.drop ('Other_last_name', 1) 
scoring = scoring.drop ('Ticket_first', 1) 
scoring = scoring.drop ('Ticket_second', 1) 


In [184]:
scoring = create_dummies(scoring,0)

In [185]:
# running on the whole dataset
y = pd.concat([y_train, y_test])
X = pd.concat([X_train, X_test])

X_scoring = scoring

In [186]:
#X.isna().sum()
X['Cabin_letter_T'] = X['Cabin_letter_T'].fillna(0)
X['Cabin_letter_F'] = X['Cabin_letter_F'].fillna(0)

X_scoring['Cabin_letter_F'] = X_scoring['Cabin_letter_F'].fillna(0)
X_scoring['Cabin_letter_T'] = 0

In [187]:
X_scoring.head()

,Age,Fare,Cabin_impact,Ticket_impact,Last_name_impact,Other_last_name_impact,Ticket_first_impact,Ticket_second_impact,Pclass_1,Pclass_2,...,Cabin_letter_A,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_F,Cabin_letter_G,MisAge_0.0,MisAge_1.0,Cabin_letter_T
PassengerId,,,,,,,,,,,,,,,,,,,,,
892,34.5,7.8292,0.0,0.000000,1.132623,0.000000,0.0,0.000000,0,0,...,0,0,0,0,0,0,0,1,0,0
893,47.0,7.0000,0.0,0.000000,0.000000,9.649966,0.0,0.000000,0,0,...,0,0,0,0,0,0,0,1,0,0
894,62.0,9.6875,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0,1,...,0,0,0,0,0,0,0,1,0,0
895,27.0,8.6625,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0,0,...,0,0,0,0,0,0,0,1,0,0
896,22.0,12.2875,0.0,9.649966,9.649966,0.000000,0.0,9.649966,0,0,...,0,0,0,0,0,0,0,1,0,0


In [188]:
best_classifier = RandomForestClassifier() 
best_classifier.fit(X, y)

y_pred = best_classifier.predict(X_scoring)
#y_pred_prob = best_classifier.predict_proba(X_scoring)

In [189]:
X_scoring['Survived'] = y_pred

submission = X_scoring[['Survived']]
submission.reset_index(inplace=True)
submission.to_csv('submission.csv', index = False, header = True)

In [190]:
submission.head()

,PassengerId,Survived
0,892,1
1,893,0
2,894,1
3,895,1
4,896,0
